In [177]:
import numpy as np
import pandas as pd

In [178]:
to_remove = []
to_combine = []

In [179]:
data = pd.read_csv('./data/scorecard/Most-Recent-Cohorts-Full.csv')
data = data.iloc[:, 0:1027]
len(data)

/home/huy/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6,9,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1379,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1412,1413,1427,1428,1431,1432,1503,1517,1532,1533,1534,1535,1536,1537,1538,1539,1540,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1567,1568,1573,1574,1575,1576,1577,1581,1582,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1604,1605,1606,1608,1610,1611,1614,1615,1616,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1636,1638,1640,1643,1644,1648,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1664,1666,1669,1670,1674,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1695,1697,1700,1701,1702,1703,1704,1705,1706,1707,1711,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1844

6694

### Initial College Information Columns (1 through 26)

Columns INSTURL and NPCURL are URL based which we won't need so I'm choosing to remove them. I'm also choosing to remove the columns SCH_DEG (which is recoded 0's and 4's from PREDDEG) and HCM2 (Schools on heightened cash monitoring).

In [180]:
to_remove.extend(['INSTURL', 'NPCURL', 'SCH_DEG', 'HCM2'])
subset = data.iloc[:, :26]
subset = subset.drop(['INSTURL', 'NPCURL', 'SCH_DEG', 'HCM2'], axis=1)

It seems most of this information is not null except for LOCALE2, which I will remove. There seems to be 505 universities that do not have a few of the specific identifiers at the end of this column list.

In [181]:
to_remove.append('LOCALE2')
subset.isna().sum()

UNITID             0
OPEID              0
OPEID6             0
INSTNM             0
CITY               0
STABBR             0
ZIP                0
ACCREDAGENCY     347
MAIN               0
NUMBRANCH          0
PREDDEG            0
HIGHDEG            0
CONTROL            0
ST_FIPS            0
REGION             0
LOCALE           505
LOCALE2         6694
LATITUDE         505
LONGITUDE        505
CCBASIC          505
CCUGPROF         505
CCSIZSET         505
dtype: int64

### Flag Columns (27 through 35)

Confirming that no colleges are both menonly and womenonly. These two columns can be combined in addition to potential other flag columns.

In [182]:
subset = data.iloc[:, 33:35]
subset.dropna(inplace=True)
subset[subset.sum(axis=1) > 1]

,MENONLY,WOMENONLY


Extracting flag columns which have simultaneously more than one flag (excluding menonly and womenonly). It seems as if these columns won't be able to be combined and should be left alone.

In [183]:
subset = data.iloc[:, 26:33]
subset.dropna(inplace=True)
subset = subset[subset.sum(axis=1) > 1]
subset

,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI
58,0.0,0.0,1.0,0.0,0.0,0.0,1.0
59,0.0,0.0,1.0,0.0,0.0,0.0,1.0
89,0.0,0.0,1.0,1.0,0.0,0.0,0.0
91,0.0,0.0,1.0,0.0,0.0,0.0,1.0
170,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
4694,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5031,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5088,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5727,0.0,0.0,0.0,0.0,1.0,1.0,0.0


### Admission Columns (37 through 61)

The following are percentages of the column which are NaN. We have an extremely high number of NaN columns and since this information is generally important for prospective college students, we will need to handle this in some way. Also note there seems to be a difference between ADM_RATE/SAT_AVG and ADM_RATE_ALL/SAT_AVG_ALL. I believe it has to do with colleges with multiple campuses. Regardless, we may not want to remove these values just yet.

In [184]:
subset = data.iloc[:, 36:61]
subset.isna().sum(axis=0) / len(subset)

ADM_RATE        0.703018
ADM_RATE_ALL    0.665073
SATVR25         0.818046
SATVR75         0.818046
SATMT25         0.818046
SATMT75         0.818046
SATWR25         0.891097
SATWR75         0.891097
SATVRMID        0.818046
SATMTMID        0.818046
SATWRMID        0.891097
ACTCM25         0.812967
ACTCM75         0.812967
ACTEN25         0.825217
ACTEN75         0.825217
ACTMT25         0.825217
ACTMT75         0.825217
ACTWR25         0.951598
ACTWR75         0.951598
ACTCMMID        0.812967
ACTENMID        0.825217
ACTMTMID        0.825217
ACTWRMID        0.951598
SAT_AVG         0.809083
SAT_AVG_ALL     0.786675
dtype: float64

When we remove rows which have all NaN columns, the percentage of NaN for admission rate all drops to less than 1%. The equivalent sat average all percentage is also the next lowest value.

In [185]:
subset = subset.dropna(how='all')
subset.isna().sum(axis=0)/len(subset)

ADM_RATE        0.117229
ADM_RATE_ALL    0.004440
SATVR25         0.459147
SATVR75         0.459147
SATMT25         0.459147
SATMT75         0.459147
SATWR25         0.676288
SATWR75         0.676288
SATVRMID        0.459147
SATMTMID        0.459147
SATWRMID        0.676288
ACTCM25         0.444050
ACTCM75         0.444050
ACTEN25         0.480462
ACTEN75         0.480462
ACTMT25         0.480462
ACTMT75         0.480462
ACTWR25         0.856128
ACTWR75         0.856128
ACTCMMID        0.444050
ACTENMID        0.480462
ACTMTMID        0.480462
ACTWRMID        0.856128
SAT_AVG         0.432504
SAT_AVG_ALL     0.365897
dtype: float64

### Academics (62 through 289)

Removing all null columns, we still retain a majority (>90%) of the data.

In [186]:
subset = data.iloc[:, 61:289]
subset = subset.dropna(how='all')
len(subset)/len(data)

0.9241111443083359

I decided to exclude all columns which were not Bachelor-specific (e.g. Award of at least 2 but less than four academic years...). First I made sure that the percentage and award columns matched up.

In [187]:
perc = [int(x[-2:]) for x in subset.iloc[:, :38].columns]
award = [int(x[3:5]) for x in subset.iloc[:, 38:].columns if 'BACHL' in x]
for num in zip(perc, award):
    if num[0] != num[1]:
        print(num)

In [188]:
for i in subset.iloc[:, 38:].columns:
    if 'BACHL' not in i:
        to_remove.append(i)

### Enrollment Columns (291 through 316)

Many of these data columns are discontinued as stated in the dictionary. I will remove all of these columns. The remaining columns are not sparse (~88% non NaN). Afterwards, I removed rows with all NaNs, leaving only 76 remaining NaN values. Also the CURROPER column indicates colleges that are not open. After all analysis, I will remove all rows with colleges that are not open.

In [189]:
subset = data.iloc[:, 290:316]
disc = ['UG', 'UGDS_WHITENH', 'UGDS_BLACKNH', 'UGDS_API', 'UGDS_AIANOLD', 'UGDS_HISPOLD', 'UG_NRA',
       'UG_UNKN', 'UG_WHITENH', 'UG_BLACKNH', 'UG_API', 'UG_AIANOLD', 'UG_HISPOLD', 'PPTUG_EF2', 'CURROPER']
subset.drop(disc, axis=1, inplace=True)
subset.dropna(how='all', inplace=True)
to_remove.extend(disc)

In [190]:
len(subset)/len(data)

0.8816850911263818

In [191]:
sum(subset.isna().sum())

76

### Cost Columns (317 through 382)

Similar to the previous section, many of these columns are discontinued - specifically for program and other related columns). The only exception is TUITIONFEE_PROG, which I added back in.

In [192]:
subset = data.iloc[:, 316:382]
disc = []

In [193]:
for i in subset.columns:
    if 'PROG' in i or 'OTHER' in i:
        disc.append(i)
disc.remove('TUITIONFEE_PROG')

In [194]:
subset.drop(disc, axis=1, inplace=True)

It seems we won't be able to drop all na columns here.

In [195]:
subset.dropna(how='all')

,NPT4_PUB,NPT4_PRIV,NPT41_PUB,NPT42_PUB,NPT43_PUB,NPT44_PUB,NPT45_PUB,NPT41_PRIV,NPT42_PRIV,NPT43_PRIV,...,NUM42_PRIV,NUM43_PRIV,NUM44_PRIV,NUM45_PRIV,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE
0,14990.0,NaN,14310.0,13849.0,18405.0,19016.0,19419.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,23053.0,NaN,10024.0,18634.0,NaN,7870.0
1,16953.0,NaN,14554.0,15526.0,18197.0,19455.0,19451.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24495.0,NaN,8568.0,20400.0,NaN,12096.0
2,NaN,8705.0,NaN,NaN,NaN,NaN,NaN,8705.0,NaN,NaN,...,0.0,0.0,0.0,0.0,14800.0,NaN,6950.0,6950.0,NaN,16403.0
3,15860.0,NaN,13010.0,13580.0,16306.0,17552.0,19231.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,23917.0,NaN,11122.0,23518.0,NaN,8275.0
4,13650.0,NaN,13531.0,13696.0,15166.0,11878.0,13573.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21866.0,NaN,11068.0,19396.0,NaN,9587.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6165.0,NaN,NaN,NaN
6690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,NaN
6691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19404.0,NaN,NaN,NaN
6692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1135.0,NaN


I confirm that for each variable with multiple institution types, the public institution average net price and private institution average net price are mutually exclusive and thus can be combined.

In [196]:
pub_priv_comp = []
for i in subset.iloc[:, :-6].columns:
    if i.split('_P')[0] not in pub_priv_comp:
        pub_priv_comp.append(i.split('_P')[0])

In [197]:
for i in range(len(pub_priv_comp)):
    comp = [pub_priv_comp[i] + s for s in ['_PUB', '_PRIV']]
    print(comp)
    print(sum(subset[comp].dropna(how='all').isna().sum(axis=1) != 1))
    to_combine.append((comp[0], comp[1]))

['NPT4_PUB', 'NPT4_PRIV']
0
['NPT41_PUB', 'NPT41_PRIV']
0
['NPT42_PUB', 'NPT42_PRIV']
0
['NPT43_PUB', 'NPT43_PRIV']
0
['NPT44_PUB', 'NPT44_PRIV']
0
['NPT45_PUB', 'NPT45_PRIV']
0
['NPT4_048_PUB', 'NPT4_048_PRIV']
0
['NPT4_3075_PUB', 'NPT4_3075_PRIV']
0
['NPT4_75UP_PUB', 'NPT4_75UP_PRIV']
0
['NUM4_PUB', 'NUM4_PRIV']
0
['NUM41_PUB', 'NUM41_PRIV']
0
['NUM42_PUB', 'NUM42_PRIV']
0
['NUM43_PUB', 'NUM43_PRIV']
0
['NUM44_PUB', 'NUM44_PRIV']
0
['NUM45_PUB', 'NUM45_PRIV']
0


### Pre-process Data

For this section I remove rows of closed universities, any row numbers after the ones I analyzed above, and all columns I chose to remove in the to_remove list. I also combine aforementioned columns.

In [198]:
data = data[data.iloc[:, 315] != 0]
data = data.iloc[:, :382]

In [201]:
for col in to_remove:
    data.drop(col, axis=1, inplace=True)

In [202]:
for cols in to_combine:
    new_col = data[[cols[0], cols[1]]].dropna(how='all').fillna(0).copy()
    new_col = new_col[cols[0]] + new_col[cols[1]]
    new_col = new_col.reindex(range(len(data)))
    data.drop([cols[0], cols[1]], axis=1, inplace=True)
    data.loc[:, cols[0].split('PUB')[0][:-1]] = new_col

In [204]:
data.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,MAIN,NUMBRANCH,...,NPT45,NPT4_048,NPT4_3075,NPT4_75UP,NUM4,NUM41,NUM42,NUM43,NUM44,NUM45
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,1,1,...,19419.0,14197.0,15322.0,19194.0,558.0,348.0,113.0,54.0,24.0,19.0
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,1,1,...,19451.0,14860.0,16942.0,19453.0,1250.0,425.0,195.0,220.0,184.0,226.0
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,1,1,...,NaN,8705.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,1,1,...,19231.0,13212.0,15045.0,18462.0,533.0,146.0,80.0,93.0,98.0,116.0
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,1,1,...,13573.0,13572.0,14120.0,12747.0,549.0,347.0,116.0,47.0,19.0,20.0


# Extracting Variables

This section is for if we wanted to directly obtain the name of the columns to keep.

In [ ]:
df = pd.read_excel('./data/college_dict.xlsx', sheet_name='Most_Recent_Inst_Cohort_Map')
df = df.iloc[0:1027,:]
df = df[~df['Variable Name'].isin(to_remove)]
df